# Install library

In [ ]:
!pip install tensorflow-gpu==1.14.0
!pip install Keras==2.2.4
!pip install efficientnet

# Import library

In [ ]:
import os
from sklearn.metrics import precision_recall_curve
import cv2
import numpy as np
import argparse
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
from keras import layers
from sklearn.model_selection import train_test_split
import pandas as pd
import pickle
import efficientnet.keras as efn
from keras import Model
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint,ReduceLROnPlateau,EarlyStopping,CSVLogger
from keras.optimizers import Adam
from keras import models
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
import seaborn as sns

# Define

In [ ]:
def new_listdir(path):
    return [f for f in os.listdir(path) if not f.startswith('.')]

def compressArr(a):
    d = np.array(a)
    d[d<12] = 0
    d[(d<=18)&(d>=12)] = 1
    d[(d<=25)&(d>=19)] = 2
    d[(d<=32)&(d>=26)] = 3
    d[(d<=39)&(d>=33)] = 4
    d[d>=40] = 5
    
    return d

In [ ]:
fb_dir = '../input/facebookdata/fbdata/fbdata'
insta_dir = '../input/instagram/ig_ava/image'
id_fb_dir = '../input/train-fb-id'
id_insta_dir = '../input/insta-pred'

fb = pd.read_csv('../input/facebookdata/age_gender_fb.csv')
fb['compress_age'] = compressArr(2019-fb.birthday_from_fb)

ma_train_dir = '../input/megaage/megaage_asian/megaage_asian/train'
ma_test_dir = '../input/megaage/megaage_asian/megaage_asian/test'
ma_age = np.array(open('../input/megaage/train_age2.txt','r').read().split()).reshape(-1,2).astype(int)
y_test_ma = np.array(open('../input/megaage/test_age2.txt','r').read().split()).reshape(-1,2).astype(int)
ma_name = np.array(open('../input/megaage/megaage_asian/megaage_asian/list/train_name.txt','r').read().split())
ma_age[:,0] = compressArr(ma_age[:,0])
X_test_ma = np.array(open('../input/megaage/megaage_asian/megaage_asian/list/test_name.txt','r').read().split())
y_test_ma[:,0] = compressArr(y_test_ma[:,0])

print(ma_age.shape)

# Prepare dataset for train

In [ ]:
fb_name1,fb_age,fb_gender,insta_name,insta_age = [],[],[],[],[]
r = np.random.RandomState(23)

with open('../input/gender-insta-pred/gender_insta_pred.pickle','rb') as f:
    gender_insta_pred = pickle.load(f)
for a in range(6):
    with open(os.path.join(id_insta_dir,'insta_ids_'+str(a)),'rb') as f:
        name = pickle.load(f) 
        for n in name:
            g = gender_insta_pred[n]
            insta_age.append([a,g])
        insta_name += name
    with open(os.path.join(id_fb_dir,'train3_ids_'+str(a)+'.pickle'),'rb') as f:
        name = pickle.load(f) 
        fb_age += [a]*len(name)
        fb_name1 += name

with open(os.path.join(id_insta_dir,'insta_ids_'+str(6)),'rb') as f:
    name = pickle.load(f) 
    insta_age += [[6,2]]*len(name)
    insta_name += name
    
fb_name2 = []
for a in range(2):
    with open(os.path.join(id_fb_dir,'gender_ids_'+str(a)+'.pickle'),'rb') as f:
        name = pickle.load(f) 
        fb_gender += [a]*len(name)
        fb_name2 += name

with open(os.path.join(id_fb_dir,'d00.pickle'),'rb') as f:
    name = pickle.load(f)[:8000]
    fb_age += [6]*len(name)
    fb_name1 += list(name)
    fb_name2 += list(name)
    fb_gender += [2]*len(name)

fb_age_d = {x:y for x,y in zip(fb_name1,fb_age)}
fb_gender_d = {x:y for x,y in zip(fb_name2,fb_gender)}
print(len(fb_name1))
print(len(fb_name2))
fb_name = list(set(fb_name1)&set(fb_name2))
fb_gender = [fb_gender_d[x] for x in fb_name]
fb_age = [fb_age_d[x] for x in fb_name]

print(len(fb_name))
fb_name = np.array(fb_name)
fb_name = np.array([str(x)+'.jpg' for x in fb_name])
fb_age = np.array(fb_age).reshape(1,-1)
fb_gender = np.array(fb_gender).reshape(1,-1)
fb_age = np.vstack((fb_age,fb_gender)).transpose()

insta_name = np.array(insta_name)
insta_age = np.array(insta_age)
#pd.Series(insta_age).value_counts().plot('bar')

In [ ]:
X_train_fb,X_test_fb,y_train_fb,y_test_fb = train_test_split(fb_name,fb_age,stratify=fb_age,test_size=0.15,random_state=23)
X_train_fb,X_val_fb,y_train_fb,y_val_fb = train_test_split(X_train_fb,y_train_fb,stratify=y_train_fb,test_size=0.15,random_state=23)

X_train_insta,X_test_insta,y_train_insta,y_test_insta = train_test_split(insta_name,insta_age,stratify=insta_age,test_size=0.15,random_state=23)
X_train_insta,X_val_insta,y_train_insta,y_val_insta = train_test_split(X_train_insta,y_train_insta,stratify=y_train_insta,test_size=0.15,random_state=23)

X_train_ma,X_val_ma,y_train_ma,y_val_ma = train_test_split(ma_name,ma_age,test_size=0.15,random_state=23)

X_val = np.array(list([os.path.join(fb_dir,x) for x in X_val_fb]) + list([os.path.join(ma_train_dir,x) for x in X_val_ma])+list(X_val_insta))
X_train = np.array(list([os.path.join(fb_dir,x) for x in X_train_fb]) + list([os.path.join(ma_train_dir,x) for x in X_train_ma])+list(X_train_insta))
y_val = np.concatenate((y_val_fb , y_val_ma,y_val_insta))
y_train = np.concatenate((y_train_fb , y_train_ma,y_train_insta))

X_test_fb = np.array(list([os.path.join(fb_dir,x) for x in X_test_fb]))
X_test_ma = np.array(list([os.path.join(ma_test_dir,x) for x in X_test_ma]))

In [ ]:
img_size=320

# Function

In [ ]:
datagen1 = ImageDataGenerator(
            width_shift_range=0.1,
            height_shift_range=0.1,
            rotation_range=15,
            #zoom_range=(0.8,1.9),
            #zoom_range=(0.8,2.5),
            brightness_range= (0.9,1.1),
            horizontal_flip=True)


def cv2_clipped_zoom(img, zoom_factor):
    height, width = img.shape[:2] # It's also the final desired shape
    new_height, new_width = int(height * zoom_factor), int(width * zoom_factor)
    y1, x1 = max(0, new_height - height) // 2, max(0, new_width - width) // 2
    y2, x2 = y1 + height, x1 + width
    bbox = np.array([y1,x1,y2,x2])
    bbox = (bbox / zoom_factor).astype(np.int)
    y1, x1, y2, x2 = bbox
    cropped_img = img[y1:y2, x1:x2]
    resize_height, resize_width = min(new_height, height), min(new_width, width)
    pad_height1, pad_width1 = (height - resize_height) // 2, (width - resize_width) //2
    pad_height2, pad_width2 = (height - resize_height) - pad_height1, (width - resize_width) - pad_width1
    pad_spec = [(pad_height1, pad_height2), (pad_width1, pad_width2)] + [(0,0)] * (img.ndim - 2)

    result = cv2.resize(cropped_img, (resize_width, resize_height),interpolation=cv2.INTER_LINEAR)
    result = np.pad(result, pad_spec, mode='constant')
    assert result.shape[0] == height and result.shape[1] == width
    return result

def generate_data(name,age,batch_size=8,is_train=True):
  while True:
    if is_train:
      indices = np.random.permutation(len(name))
    else:
      indices = np.arange(len(name))
    n_batch = int(np.ceil(len(name)/batch_size))
    j = 0
    X,y,g = [],[],[]
    for i in range(len(name)):
      j += 1
    
      filename = name[indices[i]]
      img = cv2.imread(filename)
      if filename.find('megaage')!=-1:
        h,w,c = img.shape
        t,l = (max(h,w)-h)//2,(max(h,w)-w)//2
        img = cv2.copyMakeBorder(img,t,t,l,l,cv2.BORDER_CONSTANT,value=[0,0,0])
      
        
      if is_train:
        if filename.find('megaage')!=-1:
            s = np.random.uniform(0.37,1.2)
        else:
            s = np.random.uniform(0.8,1.2)
        img = cv2_clipped_zoom(img,s)
        img = datagen1.random_transform(img)
      img = cv2.resize(img,(img_size,img_size),interpolation = cv2.INTER_AREA)
      X.append(img/255)
      y.append(age[indices[i]][0])
      g.append(age[indices[i]][1])
      if j>=batch_size:
        yield np.array(X),[np.array(y)]*5+[np.array(g)]*5
        X,y,g = [],[],[]
        j = 0
    if j >0:
      yield np.array(X),[np.array(y)]*5+[np.array(g)]*5

for X,y in generate_data(X_train,y_train,32):
    plt.imshow(X[8][:,:,(2,1,0)])
    print(y)
    break

# Build Model

In [ ]:

path = '../input/agegender320/model.h5'

model = models.load_model(path)

# Evaluate

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.utils.multiclass import unique_labels

def plot_confusion_matrix(y_true, y_pred, classes,
                          normalize=False,
                          title=None,
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """

    # Compute confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    # Only use the labels that appear in the data
    classes = classes[unique_labels(y_true, y_pred)]

#     print(cm)

    fig, ax = plt.subplots(figsize=(6,6))
    im = ax.imshow(cm, interpolation='nearest', cmap=cmap)
    ax.figure.colorbar(im, ax=ax)
    # We want to show all ticks...
    ax.set(xticks=np.arange(cm.shape[1]),
           yticks=np.arange(cm.shape[0]),
           # ... and label them with the respective list entries
           xticklabels=classes, yticklabels=classes,
           title=title,
           ylabel='True label',
           xlabel='Predicted label')

    # Rotate the tick labels and set their alignment.
    plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
             rotation_mode="anchor")

    # Loop over data dimensions and create text annotations.
    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            ax.text(j, i, format(cm[i, j], fmt),
                    ha="center", va="center",
                    color="white" if cm[i, j] > thresh else "black")
    fig.tight_layout()
    return ax, cm

def pr_curve(y_true,prob):
    # precision recall curve
    plt.figure(figsize=(8,8))
    precision = dict()
    recall = dict()
    threshold = dict()
    n_values = np.max(y_true) + 1
    y = np.eye(n_values)[y_true]
    for i in range(n_values):
        precision[i], recall[i],threshold[i] = precision_recall_curve(y[:, i],
                                                        prob[:, i])
        plt.plot(recall[i], precision[i], lw=2, label='class {}'.format(i))
    plt.xlabel("recall")
    plt.ylabel("precision")
    plt.legend(loc="best")
    plt.title("precision vs. recall curve")
    plt.show()

In [ ]:
def predict(X_test,y_test):
    y_pred_a = []
    y_pred_g = []
    
    batch_size=32
    n_test_batchs = int(np.ceil(len(X_test)/batch_size))
    i=0
    for X,y in generate_data(X_test,y_test,batch_size,is_train=False):
        p = model.predict(X)
        p_a = np.transpose(p[:5],(1,0,2))
        p_g = np.transpose(p[5:],(1,0,2))
        y_pred_a.extend(p_a)
        y_pred_g.extend(p_g)
        i+=1
        if i==n_test_batchs:
            break
    y_pred_a = np.array(y_pred_a)
    y_pred_g = np.array(y_pred_g)
    prob_a = np.mean(y_pred_a*np.reshape([1.2,1.1,1,0.9,0.8],(-1,5,1)),1)
    pred_a = np.argmax(prob_a,-1)
    prob_g = np.mean(y_pred_g*np.reshape([1.2,1.1,1,0.9,0.8],(-1,5,1)),1)
    pred_g = np.argmax(prob_g,-1)
    return pred_a,prob_a,pred_g,prob_g

In [ ]:
fb_pred_a,fb_prob_a,fb_pred_g,fb_prob_g = predict(X_test_fb,y_test_fb)

In [ ]:
import re

ids = np.array([int(re.findall('\d+',x)[0]) for x in X_test_fb])
ids[10:40]

In [ ]:
def report_a(y_true,y_pred,prob):
    
    print(classification_report(y_true,y_pred))
    _, cm = plot_confusion_matrix(y_true,y_pred, classes=np.array(["range1", "range2", "range3", "range4", "range5", "range6","undefined"]))
    plt.show()
    pr_curve(y_true,prob)
    plt.show()
    
def report_g(y_true,y_pred,prob):
    
    print(classification_report(y_true,y_pred))
    _, cm = plot_confusion_matrix(y_true, y_pred, classes=np.array(["female", "male", "undefined"]), title='Confusion matrix')
    plt.show()
    pr_curve(y_true,prob)
    plt.show()
        
report_a(y_test_fb[:,0],fb_pred_a,fb_prob_a)
report_g(y_test_fb[:,1],fb_pred_g,fb_prob_g)

In [ ]:
ma_pred_a,ma_prob_a,ma_pred_g,ma_prob_g = predict(X_test_ma,y_test_ma)

In [ ]:
report_a(y_test_ma[:,0],ma_pred_a,ma_prob_a)
report_g(y_test_ma[:,1],ma_pred_g,ma_prob_g)

In [ ]:
insta_pred_a,insta_prob_a,insta_pred_g,insta_prob_g = predict(X_test_insta,y_test_insta)
report_a(y_test_insta[:,0],insta_pred_a,insta_prob_a)
report_g(y_test_insta[:,1],insta_pred_g,insta_prob_g)

# Visualize

In [ ]:
#insta_train_pred = predict(X_train_insta,y_train_insta)
#report(y_train_insta,insta_train_pred)

In [ ]:
#Random visualize image òf ids
def visualize(ids,labels=None,size=None,n_imgs=16):
    plt.figure(figsize=(15,15))
    for k in range(n_imgs):
        i = np.random.randint(len(ids))
        id = ids[i]
        img = cv2.imread(id)
        if size:
            img = cv2.resize(img,(size,size))
        plt.subplot(4,4,k+1)
        plt.imshow(img[:,:,[2,1,0]])
        if labels is not None:
            plt.title(labels[i])

In [ ]:
label = np.array([str(x) +' vs '+str(y) for x,y in zip(y_test_insta[:,1],insta_pred_g)])
idx = y_test_insta[:,1]!=insta_pred_g
visualize(X_test_insta[idx],label[idx])

In [ ]:
label = np.array([str(x) +' vs '+str(y) for x,y in zip(y_test_insta[:,0],insta_pred_a)])
idx = y_test_insta[:,0]!=insta_pred_a
visualize(X_test_insta[idx],label[idx])

In [ ]:
#label = np.array([str(x) +' vs '+str(y) for x,y in zip(y_train_insta,insta_train_pred)])
#idx = (y_train_insta!=insta_train_pred)&(insta_train_pred==4)
#print(np.sum(idx))
#visualize(X_train_insta[idx],label[idx])

In [ ]:
label = np.array([str(x) +' vs '+str(y) for x,y in zip(y_test_fb[:,1],fb_pred_g)])
idx = y_test_fb[:,1]!=fb_pred_g
visualize(X_test_fb[idx],label[idx])

In [ ]:
label = np.array([str(x) +' vs '+str(y) for x,y in zip(y_test_fb[:,0],fb_pred_a)])
idx = y_test_fb[:,0]!=fb_pred_a
visualize(X_test_fb[idx],label[idx])

In [ ]:
label = np.array([str(x) +' vs '+str(y) for x,y in zip(compressArr(y_test_ma[:,1]),ma_pred_g)])
idx = compressArr(y_test_ma[:,1])!=ma_pred_g
visualize(X_test_ma[idx],label[idx])

In [ ]:
label = np.array([str(x) +' vs '+str(y) for x,y in zip(compressArr(y_test_ma[:,0]),ma_pred_a)])
idx = compressArr(y_test_ma[:,0])!=ma_pred_a
visualize(X_test_ma[idx],label[idx])